In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import necessary libraries
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# Import our dataset annotation module
from dataset_annotation import (
    load_helpsteer2_dataset,
    load_reward_model,
    evaluate_with_reward_model,
    save_annotated_dataset,
    load_annotated_dataset,
    annotate_dataset
)

# Set up plotting
plt.style.use('ggplot')
sns.set(style="whitegrid")
%matplotlib inline

In [3]:
# Load the reward model
model_name = "Ray2333/GRM-Gemma2-2B-rewardmodel-ft"
# model_name = "Ray2333/GRM-Llama3.2-3B-rewardmodel-ft"
dataset_save_path = "data/helpsteer2_subset_gold/"

In [4]:
model, tokenizer = load_reward_model(model_name)

Loading reward model Ray2333/GRM-Gemma2-2B-rewardmodel-ft on cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [62]:
# Define a custom dataset loader for our subset
def load_subset():
    dataset = load_helpsteer2_dataset("train")
    return dataset.select(range(100))

# Run the annotation pipeline with our custom loader
annotate_dataset(
    custom_dataset=load_subset(),  # Pass our subset
    model_name=model_name,
    batch_size=2,
    output_path=dataset_save_path
)

Loaded 9125 examples from train split
Loading reward model Ray2333/GRM-Gemma2-2B-rewardmodel-ft on cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Evaluating with reward model: 100%|██████████| 50/50 [00:26<00:00,  1.88it/s]

Reward model accuracy: 0.8000
Average reward gap: 1.4621
Min reward gap: 0.1211
Max reward gap: 3.1152
Saved annotated dataset to data/helpsteer2_subset_gold/test.json
Reward model accuracy: 0.7158
Average reward gap: 2.3215
Min reward gap: 0.0117
Max reward gap: 10.5227
Saved annotated dataset to data/helpsteer2_subset_gold/train.json


'data/helpsteer2_subset_gold/train.json'

In [63]:
# loaded_results = load_annotated_dataset(dataset_save_path)
# loaded_results

In [64]:
from reward_models.load_datasets import build_dataset
tokenizer.max_length = 1024
ds = build_dataset('./' + dataset_save_path, tokenizer=tokenizer, model_name=model_name)
ds

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

The OrderedVocab you are attempting to save contains holes for indices [0], your vocabulary could be corrupted !


Map:   0%|          | 0/95 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids_chosen', 'attention_mask_chosen', 'input_ids_rejected', 'attention_mask_rejected', 'label_chosen', 'label_rejected'],
    num_rows: 95
})

In [65]:
from datasets import load_dataset

ds = load_dataset('./' + dataset_save_path, split='train')
ds

Dataset({
    features: ['preference_strength', 'chosen', 'rejected', 'chosen_reward', 'rejected_reward', 'does_gold_agree_with_original'],
    num_rows: 95
})